In [ ]:
import psycopg2
from pgvector.psycopg2 import register_vector

conn = psycopg2.connect(
    dbname="mydatabase",
    user="myuser",
    password="mypassword",
    host="localhost",
    port="5433"
)
cur = conn.cursor()

register_vector(cur)

cur.execute("""
    CREATE TABLE IF NOT EXISTS test_vectors (
        id SERIAL PRIMARY KEY,
        name TEXT,
        embedding vector(3)  -- 这里只用 3 维向量，简单测试
    )
""")
conn.commit()

cur.execute("""
    INSERT INTO test_vectors (name, embedding)
    VALUES (%s, %s)
""", ("example", [0.1, 0.2, 0.3]))
conn.commit()

cur.execute("SELECT id, name, embedding FROM test_vectors;")
print("All stored vectors:", cur.fetchall())

cur.execute("""
    SELECT id, name, embedding
    FROM test_vectors
    ORDER BY embedding <-> %s::vector
    LIMIT 1
""", (str([0.15, 0.25, 0.35]),))

print("Most similar result:", cur.fetchone())

All stored vectors: [(1, 'example', array([0.1, 0.2, 0.3], dtype=float32)), (2, 'example', array([0.1, 0.2, 0.3], dtype=float32)), (3, 'example', array([0.1, 0.2, 0.3], dtype=float32)), (4, 'example', array([0.1, 0.2, 0.3], dtype=float32)), (5, 'example', array([0.1, 0.2, 0.3], dtype=float32))]
Most similar result: (1, 'example', array([0.1, 0.2, 0.3], dtype=float32))
